In [6]:
#import dependencies
import pandas as pd
import numpy as np

#### Extract CSVs into DataFrames

In [7]:
# Name of the CSV file
#CDC data, later updated DataFrame to use just the years 2009-2019
CDC_file = '../Resources/2005-2019-MMR-US-CDC.csv'
CDC_births = '../Resources/Births_CDC_2009_2019.csv'

# insurance files by year
ins_file_2019 = '../Resources/2019-US-health-insurance-coverage.csv'
ins_file_2017 = '../Resources/2017_US_insurance_coverage.csv'
ins_file_2015 = '../Resources/2015_US_insurance_coverage.csv'
ins_file_2013 = '../Resources/2013_US_health_insurance.csv'
ins_file_2011 = '../Resources/2011_US_ins_coverage.csv'
ins_file_2009 = '../Resources/2009_US_ins_coverage.csv'
ins_file_even = '../Resources/Even_Health_Ins_Years_Cleaned.csv'

In [8]:
# read csv files
cdc_df = pd.read_csv(CDC_file)
cdc_births_df = pd.read_csv(CDC_births, names=['State', 'State Code', 'Year', 'Births'], header=0)
ins_df = pd.read_csv(ins_file_2019, names=['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare',
       'Military', 'Uninsured', 'Total', 'Footnotes'], header=0)
ins_df_2017 = pd.read_csv(ins_file_2017, names=['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare',
       'Military', 'Uninsured', 'Total', 'Footnotes'], header=0)
ins_df_2015 = pd.read_csv(ins_file_2015, names=['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare',
       'Military', 'Uninsured', 'Total', 'Footnotes'], header=0)
ins_df_2013 = pd.read_csv(ins_file_2013, names=['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare',
       'Military', 'Uninsured', 'Total', 'Footnotes'], header=0)
ins_df_2011 = pd.read_csv(ins_file_2011, names=['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare',
       'Military', 'Uninsured', 'Total', 'Footnotes'], header=0)
ins_df_2009 = pd.read_csv(ins_file_2009, names=['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare',
       'Military', 'Uninsured', 'Total', 'Footnotes'], header=0)
ins_df_even = pd.read_csv(ins_file_even, header=0)

#### CDC MMR 2005-2019 [source](https://wonder.cdc.gov/)

In [9]:
cdc_df.head()

,State,State Code,Year,Deaths,Population,Crude Rate
0,Alabama,1,2005,12,2356423,Unreliable
1,Alabama,1,2006,10,2385480,Unreliable
2,Alabama,1,2015,12,2505795,Unreliable
3,Alabama,1,2016,35,2507714,1.4
4,Alabama,1,2017,41,2514911,1.6


In [10]:
#check column names
cdc_df.columns

Index(['State', 'State Code', 'Year', 'Deaths', 'Population', 'Crude Rate'], dtype='object')

In [11]:
#rename State column to delete extra characters
mmr_df = cdc_df.rename(columns={"ï»¿State": "State"})
mmr_df

,State,State Code,Year,Deaths,Population,Crude Rate
0,Alabama,1,2005,12,2356423,Unreliable
1,Alabama,1,2006,10,2385480,Unreliable
2,Alabama,1,2015,12,2505795,Unreliable
3,Alabama,1,2016,35,2507714,1.4
4,Alabama,1,2017,41,2514911,1.6
...,...,...,...,...,...,...
392,Wisconsin,55,2014,16,2898057,Unreliable
393,Wisconsin,55,2015,11,2903737,Unreliable
394,Wisconsin,55,2016,15,2905282,Unreliable
395,Wisconsin,55,2017,12,2912745,Unreliable


In [12]:
#check datatypes
mmr_df.dtypes

State         object
State Code     int64
Year           int64
Deaths         int64
Population     int64
Crude Rate    object
dtype: object

In [13]:
#look for missing values
mmr_df.count()

State         397
State Code    397
Year          397
Deaths        397
Population    397
Crude Rate    397
dtype: int64

In [14]:
#look at "unreliable" values in crude rate (these are used by the CDC for death counts < 20)
mmr_df ["Crude Rate"].value_counts()

Unreliable    160
0.7            41
0.6            38
0.5            33
0.8            22
0.4            18
0.9            17
1.1            17
1              15
1.2             8
1.4             7
1.3             5
1.5             4
1.6             3
2               2
1.7             2
0.3             2
1.9             1
2.3             1
1.8             1
Name: Crude Rate, dtype: int64

In [15]:
#use .loc to find just the years 2009-2019
columns = ['State', 'State Code', 'Year', 'Deaths', 'Population', 'Crude Rate']
mmr_range_df = mmr_df.loc[mmr_df["Year"] >= 2009, columns]
mmr_range_df                           

,State,State Code,Year,Deaths,Population,Crude Rate
2,Alabama,1,2015,12,2505795,Unreliable
3,Alabama,1,2016,35,2507714,1.4
4,Alabama,1,2017,41,2514911,1.6
5,Alabama,1,2018,31,2523756,1.2
6,Alabama,1,2019,35,2533668,1.4
...,...,...,...,...,...,...
392,Wisconsin,55,2014,16,2898057,Unreliable
393,Wisconsin,55,2015,11,2903737,Unreliable
394,Wisconsin,55,2016,15,2905282,Unreliable
395,Wisconsin,55,2017,12,2912745,Unreliable


In [16]:
#use groupby year to find total deaths for each year
year_totals = mmr_range_df.groupby(["Year"])

yearly_deaths = year_totals["Deaths"].sum()
yearly_deaths

Year
2009     841
2010     718
2011     832
2012     874
2013    1048
2014    1014
2015    1037
2016    1142
2017    1160
2018     874
2019    1010
Name: Deaths, dtype: int64

In [17]:
# make a new dataframe with total deaths by year
year_summary_df = pd.DataFrame({"U.S. Maternal Deaths": yearly_deaths})
year_summary_df

,U.S. Maternal Deaths
Year,
2009,841
2010,718
2011,832
2012,874
2013,1048
2014,1014
2015,1037
2016,1142
2017,1160


In [18]:
#use groupby state to find total deaths for each state from 2009-2019
state_totals = mmr_range_df.groupby(["State"])

state_deaths_df = state_totals["Deaths"].sum()
state_deaths_df

State
Alabama            154
Arizona            229
Arkansas           183
California         923
Colorado            87
Connecticut         25
Florida            666
Georgia            719
Idaho               10
Illinois           352
Indiana            374
Iowa                35
Kansas              73
Kentucky           161
Louisiana          304
Maryland           218
Massachusetts       47
Michigan           365
Minnesota          113
Mississippi        135
Missouri           273
Nevada              11
New Jersey         460
New Mexico          22
New York           702
North Carolina     289
Ohio               416
Oklahoma           203
Oregon              34
Pennsylvania       377
South Carolina     229
Tennessee          252
Texas             1473
Utah                65
Virginia           249
Washington         193
West Virginia       10
Wisconsin          119
Name: Deaths, dtype: int64

In [19]:
# CDC data for births 2009-2019
cdc_births_df

,State,State Code,Year,Births
0,Alabama,1,2009,62475
1,Alabama,1,2010,60050
2,Alabama,1,2011,59354
3,Alabama,1,2012,58448
4,Alabama,1,2013,58167
...,...,...,...,...
556,Wyoming,56,2015,7765
557,Wyoming,56,2016,7386
558,Wyoming,56,2017,6903
559,Wyoming,56,2018,6562


In [20]:
# check datatypes
cdc_births_df.dtypes

State         object
State Code     int64
Year           int64
Births         int64
dtype: object

In [21]:
# merge two CDC dataframes
CDC_data = pd.merge(mmr_range_df, cdc_births_df, how="inner", on=["State", "State Code", "Year"])
CDC_data

,State,State Code,Year,Deaths,Population,Crude Rate,Births
0,Alabama,1,2015,12,2505795,Unreliable,59657
1,Alabama,1,2016,35,2507714,1.4,59151
2,Alabama,1,2017,41,2514911,1.6,58941
3,Alabama,1,2018,31,2523756,1.2,57761
4,Alabama,1,2019,35,2533668,1.4,58615
...,...,...,...,...,...,...,...
303,Wisconsin,55,2014,16,2898057,Unreliable,67161
304,Wisconsin,55,2015,11,2903737,Unreliable,67041
305,Wisconsin,55,2016,15,2905282,Unreliable,66615
306,Wisconsin,55,2017,12,2912745,Unreliable,64975


In [22]:
# calculate Maternal Mortality Ratio (Number of maternal deaths / Number of live births) X 100,000
ratio = round(CDC_data["Deaths"] / CDC_data["Births"] * 100000, 2)
ratio

0      20.11
1      59.17
2      69.56
3      53.67
4      59.71
       ...  
303    23.82
304    16.41
305    22.52
306    18.47
307    20.55
Length: 308, dtype: float64

In [23]:
# add MMR as a column to the dataframe
CDC_data["Maternal Mortality Ratio"] = ratio
CDC_data

,State,State Code,Year,Deaths,Population,Crude Rate,Births,Maternal Mortality Ratio
0,Alabama,1,2015,12,2505795,Unreliable,59657,20.11
1,Alabama,1,2016,35,2507714,1.4,59151,59.17
2,Alabama,1,2017,41,2514911,1.6,58941,69.56
3,Alabama,1,2018,31,2523756,1.2,57761,53.67
4,Alabama,1,2019,35,2533668,1.4,58615,59.71
...,...,...,...,...,...,...,...,...
303,Wisconsin,55,2014,16,2898057,Unreliable,67161,23.82
304,Wisconsin,55,2015,11,2903737,Unreliable,67041,16.41
305,Wisconsin,55,2016,15,2905282,Unreliable,66615,22.52
306,Wisconsin,55,2017,12,2912745,Unreliable,64975,18.47


In [24]:
# select and reorder columns for final dataframe on CDC data
CDC_final_df = CDC_data[["State", "State Code", "Year", "Deaths", "Births", "Maternal Mortality Ratio", "Population"]]
CDC_final_df

,State,State Code,Year,Deaths,Births,Maternal Mortality Ratio,Population
0,Alabama,1,2015,12,59657,20.11,2505795
1,Alabama,1,2016,35,59151,59.17,2507714
2,Alabama,1,2017,41,58941,69.56,2514911
3,Alabama,1,2018,31,57761,53.67,2523756
4,Alabama,1,2019,35,58615,59.71,2533668
...,...,...,...,...,...,...,...
303,Wisconsin,55,2014,16,67161,23.82,2898057
304,Wisconsin,55,2015,11,67041,16.41,2903737
305,Wisconsin,55,2016,15,66615,22.52,2905282
306,Wisconsin,55,2017,12,64975,18.47,2912745


In [25]:
#rename columns for database use
CDC_final_columns_df = CDC_final_df.rename(columns={"State": "state",
                                                    "State Code": "state_code", 
                                                    "Year": "year",
                                                    "Deaths": "deaths",
                                                    "Births": "births",
                                                    "Maternal Mortality Ratio": "maternal_mortality_ratio",
                                                    "Population": "population"})
CDC_final_columns_df

,state,state_code,year,deaths,births,maternal_mortality_ratio,population
0,Alabama,1,2015,12,59657,20.11,2505795
1,Alabama,1,2016,35,59151,59.17,2507714
2,Alabama,1,2017,41,58941,69.56,2514911
3,Alabama,1,2018,31,57761,53.67,2523756
4,Alabama,1,2019,35,58615,59.71,2533668
...,...,...,...,...,...,...,...
303,Wisconsin,55,2014,16,67161,23.82,2898057
304,Wisconsin,55,2015,11,67041,16.41,2903737
305,Wisconsin,55,2016,15,66615,22.52,2905282
306,Wisconsin,55,2017,12,64975,18.47,2912745


#### US Insurance Coverage 2019 : Health Insurance Coverage of Females 19-64 [source](https://www.kff.org/other/state-indicator/health-insurance-coverage-of-nonelderly-adult-females/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

In [26]:
ins_df.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.607,0.079,0.165,0.020,0.015,0.114,1.0,1.0
1,Alabama,0.588,0.078,0.139,0.038,0.024,0.133,1.0,NaN
2,Alaska,0.591,0.041,0.170,0.013,0.065,0.119,1.0,NaN
3,Arizona,0.568,0.069,0.188,0.019,0.017,0.138,1.0,NaN
4,Arkansas,0.536,0.076,0.226,0.035,0.015,0.113,1.0,NaN


In [27]:
# check datatypes
ins_df.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military      object
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [28]:
ins_df.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.607,0.079,0.165,0.020,0.015,0.114,1.0,1.0
1,Alabama,0.588,0.078,0.139,0.038,0.024,0.133,1.0,NaN
2,Alaska,0.591,0.041,0.170,0.013,0.065,0.119,1.0,NaN
3,Arizona,0.568,0.069,0.188,0.019,0.017,0.138,1.0,NaN
4,Arkansas,0.536,0.076,0.226,0.035,0.015,0.113,1.0,NaN


In [29]:
# check column names (previously found extra space preceding column names)
ins_df.columns

Index(['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare', 'Military',
       'Uninsured', 'Total', 'Footnotes'],
      dtype='object')

In [30]:
ins_clean_df = ins_df.loc[ins_df["Military"]==" N/A"]
ins_clean_df

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
46,Vermont,0.614,0.069,0.242,0.019,N/A,0.046,1.0,NaN


In [31]:
#remove "N/A" from Military column 
ins_clean_df = ins_df.replace({" N/A": 0})
ins_clean_df.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.607,0.079,0.165,0.020,0.015,0.114,1.0,1.0
1,Alabama,0.588,0.078,0.139,0.038,0.024,0.133,1.0,NaN
2,Alaska,0.591,0.041,0.170,0.013,0.065,0.119,1.0,NaN
3,Arizona,0.568,0.069,0.188,0.019,0.017,0.138,1.0,NaN
4,Arkansas,0.536,0.076,0.226,0.035,0.015,0.113,1.0,NaN


In [32]:
# check datatypes
ins_clean_df.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military      object
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [33]:
# use pd.to_numeric() method to convert the datatype of the Military column from object to float
ins_clean_df["Military"] = pd.to_numeric(ins_clean_df["Military"])

In [34]:
# verify data types 
ins_clean_df.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military     float64
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [35]:
#convert values to percentages and then use .style.format({"column name" : "{:,.2f}%"})
employer_per = ins_clean_df["Employer"] * 100
employer_per

non_group_per = ins_clean_df["Non-Group"] * 100
non_group_per

medicaid_per = ins_clean_df["Medicaid"] * 100
medicaid_per

medicare_per = ins_clean_df["Medicare"] * 100
medicare_per

military_per = ins_clean_df["Military"] * 100
military_per

uninsured_per = ins_clean_df["Uninsured"] * 100
uninsured_per

total_per = ins_clean_df["Total"] * 100
total_per.head()

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
Name: Total, dtype: float64

In [36]:
# assign locations to list for new dataframe
locations = ins_clean_df["Location"]
locations.head()

0     United States
1           Alabama
2            Alaska
3           Arizona
4          Arkansas
Name: Location, dtype: object

In [37]:
# make a new dataframe with percentages and without the footnotes column
us_ins_2019_df = pd.DataFrame({'Location': locations,
                               'Year': 2019,
                               'Employer': employer_per, 
                               'Non-Group': non_group_per,
                               'Medicaid': medicaid_per,
                               'Medicare': medicare_per, 
                               'Military': military_per,
                               'Uninsured': uninsured_per, 
                               'Total': total_per})
us_ins_2019_df.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2019,60.7,7.9,16.5,2.0,1.5,11.4,100.0
1,Alabama,2019,58.8,7.8,13.9,3.8,2.4,13.3,100.0
2,Alaska,2019,59.1,4.1,17.0,1.3,6.5,11.9,100.0
3,Arizona,2019,56.8,6.9,18.8,1.9,1.7,13.8,100.0
4,Arkansas,2019,53.6,7.6,22.6,3.5,1.5,11.3,100.0


#### US insurance coverage 2013 : Health Insurance Coverage of Females 19-64 [source](https://www.kff.org/other/state-indicator/health-insurance-coverage-of-nonelderly-adult-females/?currentTimeframe=6&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

In [38]:
ins_df_2013.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.581,0.067,0.129,0.020,0.016,0.187,1.0,1.0
1,Alabama,0.555,0.068,0.125,0.041,0.021,0.190,1.0,NaN
2,Alaska,0.600,0.037,0.075,NaN,0.057,0.223,1.0,NaN
3,Arizona,0.527,0.072,0.151,0.018,0.018,0.213,1.0,NaN
4,Arkansas,0.528,0.063,0.123,0.037,0.020,0.229,1.0,NaN


In [39]:
# check datatypes
ins_df_2013.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military     float64
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [40]:
#check column names (previously found extra space preceding column names)
ins_df_2013.columns

Index(['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare', 'Military',
       'Uninsured', 'Total', 'Footnotes'],
      dtype='object')

In [41]:
clean_ins_df_2013 = ins_df_2013.fillna(0)
clean_ins_df_2013.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.581,0.067,0.129,0.020,0.016,0.187,1.0,1.0
1,Alabama,0.555,0.068,0.125,0.041,0.021,0.190,1.0,0.0
2,Alaska,0.600,0.037,0.075,0.000,0.057,0.223,1.0,0.0
3,Arizona,0.527,0.072,0.151,0.018,0.018,0.213,1.0,0.0
4,Arkansas,0.528,0.063,0.123,0.037,0.020,0.229,1.0,0.0


In [42]:
#convert values to percentages and then use .style.format({"column name" : "{:,.2f}%"})
employer_per_2013 = clean_ins_df_2013["Employer"] * 100
employer_per_2013

non_group_per_2013 = clean_ins_df_2013["Non-Group"] * 100
non_group_per_2013

medicaid_per_2013 = clean_ins_df_2013["Medicaid"] * 100
medicaid_per_2013

medicare_per_2013 = clean_ins_df_2013["Medicare"] * 100
medicare_per_2013

military_per_2013 = clean_ins_df_2013["Military"] * 100
military_per_2013

uninsured_per_2013 = clean_ins_df_2013["Uninsured"] * 100
uninsured_per_2013

total_per_2013 = clean_ins_df_2013["Total"] * 100
total_per_2013.head()

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
Name: Total, dtype: float64

In [43]:
# assign locations to list for new dataframe
locations_2013 = clean_ins_df_2013["Location"]
locations_2013.head()

0    United States
1          Alabama
2           Alaska
3          Arizona
4         Arkansas
Name: Location, dtype: object

In [44]:
# make a new dataframe with percentages and without the footnotes column
us_ins_2013_df = pd.DataFrame({'Location': locations_2013,
                               'Year': 2013,
                               'Employer': employer_per_2013, 
                               'Non-Group': non_group_per_2013,
                               'Medicaid': medicaid_per_2013,
                               'Medicare': medicare_per_2013, 
                               'Military': military_per_2013,
                               'Uninsured': uninsured_per_2013, 
                               'Total': total_per_2013})
us_ins_2013_df.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2013,58.1,6.7,12.9,2.0,1.6,18.7,100.0
1,Alabama,2013,55.5,6.8,12.5,4.1,2.1,19.0,100.0
2,Alaska,2013,60.0,3.7,7.5,0.0,5.7,22.3,100.0
3,Arizona,2013,52.7,7.2,15.1,1.8,1.8,21.3,100.0
4,Arkansas,2013,52.8,6.3,12.3,3.7,2.0,22.9,100.0


#### US insurance coverage 2009 : Health Insurance Coverage of Females 19-64 [source](https://www.kff.org/other/state-indicator/health-insurance-coverage-of-nonelderly-adult-females/?currentTimeframe=10&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

In [45]:
ins_df_2009.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.603,0.066,0.115,0.019,0.014,0.183,1.0,1.0
1,Alabama,0.586,0.061,0.114,0.035,0.022,0.182,1.0,NaN
2,Alaska,0.567,0.042,0.101,NaN,0.056,0.227,1.0,NaN
3,Arizona,0.538,0.069,0.155,0.020,0.017,0.201,1.0,NaN
4,Arkansas,0.541,0.058,0.118,0.036,0.015,0.231,1.0,NaN


In [46]:
# check datatypes
ins_df_2009.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military     float64
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [47]:
# check column names
ins_df_2009.columns

Index(['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare', 'Military',
       'Uninsured', 'Total', 'Footnotes'],
      dtype='object')

In [48]:
# adjust NaN values to 0 with .fillna(0)
ins_df_2009_clean = ins_df_2009.fillna(0)
ins_df_2009_clean.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.603,0.066,0.115,0.019,0.014,0.183,1.0,1.0
1,Alabama,0.586,0.061,0.114,0.035,0.022,0.182,1.0,0.0
2,Alaska,0.567,0.042,0.101,0.000,0.056,0.227,1.0,0.0
3,Arizona,0.538,0.069,0.155,0.020,0.017,0.201,1.0,0.0
4,Arkansas,0.541,0.058,0.118,0.036,0.015,0.231,1.0,0.0


In [49]:
#convert values to percentages 

employer_per_2009 = ins_df_2009_clean["Employer"] * 100
employer_per_2009

non_group_per_2009 = ins_df_2009_clean["Non-Group"] * 100
non_group_per_2009

medicaid_per_2009 = ins_df_2009_clean["Medicaid"] * 100
medicaid_per_2009

medicare_per_2009 = ins_df_2009_clean["Medicare"] * 100
medicare_per_2009

military_per_2009 = ins_df_2009_clean["Military"] * 100
military_per_2009

uninsured_per_2009 = ins_df_2009_clean["Uninsured"] * 100
uninsured_per_2009

total_per_2009 = ins_df_2009_clean["Total"] * 100
total_per_2009.head()

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
Name: Total, dtype: float64

In [50]:
# generate list of locations for building new dataframe
locations_2009 = ins_df_2009_clean["Location"]
locations_2009.head()

0    United States
1          Alabama
2           Alaska
3          Arizona
4         Arkansas
Name: Location, dtype: object

In [51]:
# make a new dataframe with percentages and without the footnotes column
us_ins_2009_df = pd.DataFrame({'Location': locations_2009,
                               'Year': 2009,
                               'Employer': employer_per_2009, 
                               'Non-Group': non_group_per_2009,
                               'Medicaid': medicaid_per_2009,
                               'Medicare': medicare_per_2009, 
                               'Military': military_per_2009,
                               'Uninsured': uninsured_per_2009, 
                               'Total': total_per_2009})
us_ins_2009_df.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2009,60.3,6.6,11.5,1.9,1.4,18.3,100.0
1,Alabama,2009,58.6,6.1,11.4,3.5,2.2,18.2,100.0
2,Alaska,2009,56.7,4.2,10.1,0.0,5.6,22.7,100.0
3,Arizona,2009,53.8,6.9,15.5,2.0,1.7,20.1,100.0
4,Arkansas,2009,54.1,5.8,11.8,3.6,1.5,23.1,100.0


#### US insurance coverage 2011 : Health Insurance Coverage of Females 19-64 [source](https://www.kff.org/other/state-indicator/health-insurance-coverage-of-nonelderly-adult-females/?currentTimeframe=8&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

In [52]:
ins_df_2011.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.586,0.064,0.125,0.019,0.015,0.191,1.0,1.0
1,Alabama,0.569,0.066,0.113,0.037,0.023,0.192,1.0,NaN
2,Alaska,0.574,0.028,0.113,NaN,0.050,0.227,1.0,NaN
3,Arizona,0.520,0.067,0.173,0.020,0.018,0.202,1.0,NaN
4,Arkansas,0.532,0.056,0.120,0.034,0.016,0.241,1.0,NaN


In [53]:
# check datatypes
ins_df_2011.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military     float64
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [54]:
# check column names
ins_df_2011.columns

Index(['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare', 'Military',
       'Uninsured', 'Total', 'Footnotes'],
      dtype='object')

In [55]:
# adjust NaN values to 0 with .fillna(0)
ins_df_2011_clean = ins_df_2011.fillna(0)
ins_df_2011_clean.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.586,0.064,0.125,0.019,0.015,0.191,1.0,1.0
1,Alabama,0.569,0.066,0.113,0.037,0.023,0.192,1.0,0.0
2,Alaska,0.574,0.028,0.113,0.000,0.050,0.227,1.0,0.0
3,Arizona,0.520,0.067,0.173,0.020,0.018,0.202,1.0,0.0
4,Arkansas,0.532,0.056,0.120,0.034,0.016,0.241,1.0,0.0


In [56]:
#convert values to percentages 

employer_per_2011 = ins_df_2011_clean["Employer"] * 100

non_group_per_2011 = ins_df_2011_clean["Non-Group"] * 100

medicaid_per_2011 = ins_df_2011_clean["Medicaid"] * 100

medicare_per_2011 = ins_df_2011_clean["Medicare"] * 100

military_per_2011 = ins_df_2011_clean["Military"] * 100

uninsured_per_2011 = ins_df_2011_clean["Uninsured"] * 100

total_per_2011 = ins_df_2011_clean["Total"] * 100
total_per_2011.head()

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
Name: Total, dtype: float64

In [57]:
# generate list of locations for building new dataframe
locations_2011 = ins_df_2011_clean["Location"]
locations_2011.head()

0    United States
1          Alabama
2           Alaska
3          Arizona
4         Arkansas
Name: Location, dtype: object

In [58]:
# make a new dataframe with percentages and without the footnotes column
us_ins_2011_df = pd.DataFrame({'Location': locations_2011,
                               'Year': 2011,
                               'Employer': employer_per_2011, 
                               'Non-Group': non_group_per_2011,
                               'Medicaid': medicaid_per_2011,
                               'Medicare': medicare_per_2011, 
                               'Military': military_per_2011,
                               'Uninsured': uninsured_per_2011, 
                               'Total': total_per_2011})
us_ins_2011_df.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2011,58.6,6.4,12.5,1.9,1.5,19.1,100.0
1,Alabama,2011,56.9,6.6,11.3,3.7,2.3,19.2,100.0
2,Alaska,2011,57.4,2.8,11.3,0.0,5.0,22.7,100.0
3,Arizona,2011,52.0,6.7,17.3,2.0,1.8,20.2,100.0
4,Arkansas,2011,53.2,5.6,12.0,3.4,1.6,24.1,100.0


#### US insurance coverage 2015 : Health Insurance Coverage of Females 19-64 [source](https://www.kff.org/other/state-indicator/health-insurance-coverage-of-nonelderly-adult-females/?currentTimeframe=4&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

In [59]:
ins_df_2015.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.588,0.092,0.167,0.021,0.015,0.116,1.0,1.0
1,Alabama,0.574,0.089,0.136,0.040,0.022,0.139,1.0,NaN
2,Alaska,0.551,0.069,0.140,0.015,0.056,0.168,1.0,NaN
3,Arizona,0.534,0.094,0.206,0.020,0.018,0.128,1.0,NaN
4,Arkansas,0.532,0.098,0.203,0.034,0.013,0.119,1.0,NaN


In [60]:
# check datatypes
ins_df_2015.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military     float64
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [61]:
# check column names
ins_df_2015.columns

Index(['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare', 'Military',
       'Uninsured', 'Total', 'Footnotes'],
      dtype='object')

In [62]:
# adjust NaN values to 0 with .fillna(0)
ins_df_2015_clean = ins_df_2015.fillna(0)
ins_df_2015_clean.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.588,0.092,0.167,0.021,0.015,0.116,1.0,1.0
1,Alabama,0.574,0.089,0.136,0.040,0.022,0.139,1.0,0.0
2,Alaska,0.551,0.069,0.140,0.015,0.056,0.168,1.0,0.0
3,Arizona,0.534,0.094,0.206,0.020,0.018,0.128,1.0,0.0
4,Arkansas,0.532,0.098,0.203,0.034,0.013,0.119,1.0,0.0


In [63]:
#convert values to percentages 

employer_per_2015 = ins_df_2015_clean["Employer"] * 100

non_group_per_2015 = ins_df_2015_clean["Non-Group"] * 100

medicaid_per_2015 = ins_df_2015_clean["Medicaid"] * 100

medicare_per_2015 = ins_df_2015_clean["Medicare"] * 100

military_per_2015 = ins_df_2015_clean["Military"] * 100

uninsured_per_2015 = ins_df_2015_clean["Uninsured"] * 100

total_per_2015 = ins_df_2015_clean["Total"] * 100
total_per_2015.head()

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
Name: Total, dtype: float64

In [64]:
# generate list of locations for building new dataframe
locations_2015 = ins_df_2015_clean["Location"]
locations_2015.head()

0    United States
1          Alabama
2           Alaska
3          Arizona
4         Arkansas
Name: Location, dtype: object

In [65]:
# make a new dataframe with percentages and without the footnotes column
us_ins_2015_df = pd.DataFrame({'Location': locations_2015,
                               'Year': 2015,
                               'Employer': employer_per_2015, 
                               'Non-Group': non_group_per_2015,
                               'Medicaid': medicaid_per_2015,
                               'Medicare': medicare_per_2015, 
                               'Military': military_per_2015,
                               'Uninsured': uninsured_per_2015, 
                               'Total': total_per_2015})
us_ins_2015_df.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2015,58.8,9.2,16.7,2.1,1.5,11.6,100.0
1,Alabama,2015,57.4,8.9,13.6,4.0,2.2,13.9,100.0
2,Alaska,2015,55.1,6.9,14.0,1.5,5.6,16.8,100.0
3,Arizona,2015,53.4,9.4,20.6,2.0,1.8,12.8,100.0
4,Arkansas,2015,53.2,9.8,20.3,3.4,1.3,11.9,100.0


#### US insurance coverage 2017 : Health Insurance Coverage of Females 19-64 [source](https://www.kff.org/other/state-indicator/health-insurance-coverage-of-nonelderly-adult-females/?currentTimeframe=2&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D)

In [66]:
ins_df_2017.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.598,0.087,0.171,0.021,0.015,0.108,1.0,1.0
1,Alabama,0.580,0.086,0.136,0.042,0.023,0.133,1.0,NaN
2,Alaska,0.547,0.048,0.195,NaN,0.062,0.143,1.0,NaN
3,Arizona,0.559,0.078,0.208,0.018,0.018,0.120,1.0,NaN
4,Arkansas,0.539,0.087,0.220,0.037,0.018,0.099,1.0,NaN


In [67]:
# check datatypes
ins_df_2017.dtypes

Location      object
Employer     float64
Non-Group    float64
Medicaid     float64
Medicare     float64
Military     float64
Uninsured    float64
Total        float64
Footnotes    float64
dtype: object

In [68]:
# check column names
ins_df_2017.columns

Index(['Location', 'Employer', 'Non-Group', 'Medicaid', 'Medicare', 'Military',
       'Uninsured', 'Total', 'Footnotes'],
      dtype='object')

In [69]:
# adjust NaN values to 0 with .fillna(0)
ins_df_2017_clean = ins_df_2017.fillna(0)
ins_df_2017_clean.head()

,Location,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total,Footnotes
0,United States,0.598,0.087,0.171,0.021,0.015,0.108,1.0,1.0
1,Alabama,0.580,0.086,0.136,0.042,0.023,0.133,1.0,0.0
2,Alaska,0.547,0.048,0.195,0.000,0.062,0.143,1.0,0.0
3,Arizona,0.559,0.078,0.208,0.018,0.018,0.120,1.0,0.0
4,Arkansas,0.539,0.087,0.220,0.037,0.018,0.099,1.0,0.0


In [70]:
#convert values to percentages 

employer_per_2017 = ins_df_2017_clean["Employer"] * 100

non_group_per_2017 = ins_df_2017_clean["Non-Group"] * 100

medicaid_per_2017 = ins_df_2017_clean["Medicaid"] * 100

medicare_per_2017 = ins_df_2017_clean["Medicare"] * 100

military_per_2017 = ins_df_2017_clean["Military"] * 100

uninsured_per_2017 = ins_df_2017_clean["Uninsured"] * 100

total_per_2017 = ins_df_2017_clean["Total"] * 100
total_per_2017.head()

0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
Name: Total, dtype: float64

In [71]:
# generate list of locations for building new dataframe
locations_2017 = ins_df_2017_clean["Location"]
locations_2017.head()

0    United States
1          Alabama
2           Alaska
3          Arizona
4         Arkansas
Name: Location, dtype: object

In [72]:
# make a new dataframe with percentages and without the footnotes column
us_ins_2017_df = pd.DataFrame({'Location': locations_2017,
                               'Year': 2017,
                               'Employer': employer_per_2017, 
                               'Non-Group': non_group_per_2017,
                               'Medicaid': medicaid_per_2017,
                               'Medicare': medicare_per_2017, 
                               'Military': military_per_2017,
                               'Uninsured': uninsured_per_2017, 
                               'Total': total_per_2017})
us_ins_2017_df.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2017,59.8,8.7,17.1,2.1,1.5,10.8,100.0
1,Alabama,2017,58.0,8.6,13.6,4.2,2.3,13.3,100.0
2,Alaska,2017,54.7,4.8,19.5,0.0,6.2,14.3,100.0
3,Arizona,2017,55.9,7.8,20.8,1.8,1.8,12.0,100.0
4,Arkansas,2017,53.9,8.7,22.0,3.7,1.8,9.9,100.0


In [73]:
# use concat() to combine insurance dataframes for odd years 2009-2019
us_ins_odd_df = pd.concat([us_ins_2009_df, us_ins_2011_df, us_ins_2013_df, us_ins_2015_df, us_ins_2017_df, us_ins_2019_df])
us_ins_odd_df.reset_index(drop=True, inplace=True)
us_ins_odd_df

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2009,60.3,6.6,11.5,1.9,1.4,18.3,100.0
1,Alabama,2009,58.6,6.1,11.4,3.5,2.2,18.2,100.0
2,Alaska,2009,56.7,4.2,10.1,0.0,5.6,22.7,100.0
3,Arizona,2009,53.8,6.9,15.5,2.0,1.7,20.1,100.0
4,Arkansas,2009,54.1,5.8,11.8,3.6,1.5,23.1,100.0
...,...,...,...,...,...,...,...,...,...
313,Washington,2019,64.0,7.0,16.9,1.7,2.2,8.2,100.0
314,West Virginia,2019,56.6,3.5,27.0,3.6,1.2,8.0,100.0
315,Wisconsin,2019,69.1,7.6,14.5,1.3,0.8,6.7,100.0
316,Wyoming,2019,60.4,9.8,9.7,1.4,1.9,16.7,100.0


In [74]:
# read in Akilah's cleaned even years 
ins_df_even_final = ins_df_even[["Location", "Year", "Employer", "Non-Group", "Medicaid", "Medicare", "Military", "Uninsured", "Total"]]
ins_df_even_final

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2010,58.9,6.5,12.1,1.8,1.5,19.2,100
1,Alabama,2010,57.5,6.3,11.5,3.2,2.1,19.4,100
2,Alaska,2010,58.1,3.2,10.5,1.4,6.3,20.5,100
3,Arizona,2010,52.9,6.5,17.1,1.6,1.8,20.1,100
4,Arkansas,2010,52.2,5.8,11.7,4.1,1.9,24.4,100
...,...,...,...,...,...,...,...,...,...
260,Washington,2018,63.0,7.4,17.8,1.5,2.2,8.0,100
261,West Virginia,2018,56.0,4.6,26.6,3.7,1.1,8.0,100
262,Wisconsin,2018,68.5,7.8,14.4,1.6,0.7,7.0,100
263,Wyoming,2018,64.0,11.1,8.2,1.4,2.1,13.2,100


In [75]:
# combining even and odd years 
us_ins_all = pd.concat([us_ins_odd_df, ins_df_even_final])
us_ins_all.reset_index(drop=True, inplace=True)
us_ins_all

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,United States,2009,60.3,6.6,11.5,1.9,1.4,18.3,100.0
1,Alabama,2009,58.6,6.1,11.4,3.5,2.2,18.2,100.0
2,Alaska,2009,56.7,4.2,10.1,0.0,5.6,22.7,100.0
3,Arizona,2009,53.8,6.9,15.5,2.0,1.7,20.1,100.0
4,Arkansas,2009,54.1,5.8,11.8,3.6,1.5,23.1,100.0
...,...,...,...,...,...,...,...,...,...
578,Washington,2018,63.0,7.4,17.8,1.5,2.2,8.0,100.0
579,West Virginia,2018,56.0,4.6,26.6,3.7,1.1,8.0,100.0
580,Wisconsin,2018,68.5,7.8,14.4,1.6,0.7,7.0,100.0
581,Wyoming,2018,64.0,11.1,8.2,1.4,2.1,13.2,100.0


In [76]:
# sorting the dataframe by year
us_ins_all_sorted = us_ins_all.sort_values(["Year", "Location"], ascending=True)
us_ins_all_sorted.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
1,Alabama,2009,58.6,6.1,11.4,3.5,2.2,18.2,100.0
2,Alaska,2009,56.7,4.2,10.1,0.0,5.6,22.7,100.0
3,Arizona,2009,53.8,6.9,15.5,2.0,1.7,20.1,100.0
4,Arkansas,2009,54.1,5.8,11.8,3.6,1.5,23.1,100.0
5,California,2009,55.4,8.6,12.2,1.3,0.9,21.6,100.0


In [77]:
# reset index for new sorted order 
us_ins_all_sorted_index = us_ins_all_sorted.reset_index(drop=True)
us_ins_all_sorted_index.head()

,Location,Year,Employer,Non-Group,Medicaid,Medicare,Military,Uninsured,Total
0,Alabama,2009,58.6,6.1,11.4,3.5,2.2,18.2,100.0
1,Alaska,2009,56.7,4.2,10.1,0.0,5.6,22.7,100.0
2,Arizona,2009,53.8,6.9,15.5,2.0,1.7,20.1,100.0
3,Arkansas,2009,54.1,5.8,11.8,3.6,1.5,23.1,100.0
4,California,2009,55.4,8.6,12.2,1.3,0.9,21.6,100.0


In [78]:
# rename columns for loading into database
us_ins_all_rename = us_ins_all_sorted_index.rename(columns={"Location": "location",
                                                            "Year": "year",
                                                            "Employer": "employer",
                                                            "Non-Group": "non_group",
                                                            "Medicaid": "medicaid",
                                                            "Medicare": "medicare",
                                                            "Military": "military",
                                                            "Uninsured": "uninsured",
                                                            "Total": "total"})
us_ins_all_rename

,location,year,employer,non_group,medicaid,medicare,military,uninsured,total
0,Alabama,2009,58.6,6.1,11.4,3.5,2.2,18.2,100.0
1,Alaska,2009,56.7,4.2,10.1,0.0,5.6,22.7,100.0
2,Arizona,2009,53.8,6.9,15.5,2.0,1.7,20.1,100.0
3,Arkansas,2009,54.1,5.8,11.8,3.6,1.5,23.1,100.0
4,California,2009,55.4,8.6,12.2,1.3,0.9,21.6,100.0
...,...,...,...,...,...,...,...,...,...
578,Virginia,2019,65.7,7.0,10.7,2.4,4.9,9.4,100.0
579,Washington,2019,64.0,7.0,16.9,1.7,2.2,8.2,100.0
580,West Virginia,2019,56.6,3.5,27.0,3.6,1.2,8.0,100.0
581,Wisconsin,2019,69.1,7.6,14.5,1.3,0.8,6.7,100.0


In [81]:
# Export file as a CSV, without the Pandas index, but with the header
us_ins_all_rename.to_csv("../output_file/2009_2019_US_ins_cleaned.csv", index=False, header=True, float_format='%.2f')

In [82]:
# export CDC DataFrame as a csv
CDC_final_columns_df.to_csv("../output_file/2009_2019_CDC_MMR.csv", index=False, header=True)